(complex-queries)=
# More Complex Queries

## Introduction

The queries represented in the following tutorial are as follows:
* [**All the Stops and Arrival Times for an Operator on a Given Day**](stop-arrivals-operator)
* [**Filter with Lists and Unpacking**](#filter-with-lists-and-unpacking)

## Python Libraries to Import

In [1]:
import os
import pandas as pd

os.environ["CALITP_BQ_MAX_BYTES"] = str(50_000_000_000)

import calitp
from calitp.tables import tbls
from siuba import *

pd.set_option("display.max_rows", 10)

SELECTED_DATE = "2021-09-01"
ITP_ID = 208 # Monterey-Salinas Transit

/opt/hostedtoolcache/Python/3.8.14/x64/lib/python3.8/site-packages/pkg_resources/__init__.py:2456: UserWarning: Obsolete pybigquery is installed, which is likely to
interfere with sqlalchemy_bigquery.
pybigquery should be uninstalled.
  module = __import__(self.module_name, fromlist=['__name__'], level=0)


(stop-arrivals-operator)=
### All the Stops and Arrival Times for an Operator on a Given Day

As a simple example, we will filter to just Monterey-Salinas Transit and grab 1 day's worth of data. We want all the trips, stops, arrival times, and stop geometry (lat/lon).

Tables used:
1. `views.gtfs_schedule_dim_stop_times`: all stop arrival times for all operators, need to subset to particular date
1. `views.gtfs_schedule_fact_daily_trips`: all trips for all operators, need to subset to particular date
1. `views.gtfs_schedule_dim_stops`: lat/lon for all stops, need to subset to interested stops

Here, all the trips for one operator, for a particular day, is joined with all the stops that occur on all the trips. Then, the stops have their lat/lon information attached.

In [2]:
## Get trips for operator for one day and join with stop times for all trips

# Grab the stop times for a given date for just 1 agency
tbl_stop_times = (
    tbls.views.gtfs_schedule_dim_stop_times()
    >> filter(_.calitp_extracted_at <= SELECTED_DATE,
              _.calitp_deleted_at > SELECTED_DATE,
              _.calitp_itp_id == ITP_ID
             )
)

# Grab the trips done on that day, for that agency
daily_stops = (
    tbls.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.calitp_itp_id == ITP_ID,
              _.service_date == SELECTED_DATE,
              _.is_in_service == True)
    # Join the trip to the stop time
    # For a given bus route (left df), attach all the stops (right df)
    >> left_join(_, tbl_stop_times,
              # also added url number to the join keys ----
             ["calitp_itp_id", "calitp_url_number", "trip_id"])
    >> inner_join(_, tbls.views.gtfs_schedule_dim_stops(),
                 ["calitp_itp_id", "stop_id"])
    >> select(_.itp_id == _.calitp_itp_id,
              _.date == _.service_date,
              _.trip_key, _.trip_id, _.stop_id, _.arrival_time,
              _.stop_lat, _.stop_lon, _.stop_name,
             )
    >> collect()
    )

daily_stops.head()

itp_id        date            trip_key   trip_id stop_id arrival_time  \
0     208  2021-09-01  325445739163806226  95030002    1602     10:12:00   
1     208  2021-09-01  325445739163806226  95030002    1602     10:12:00   
2     208  2021-09-01  325445739163806226  95030002    1602     10:12:00   
3     208  2021-09-01  325445739163806226  95030002    1602     10:12:00   
4     208  2021-09-01  325445739163806226  95030002    1602     10:12:00   

    stop_lat    stop_lon        stop_name  
0  36.609972 -121.897399  Foam /  Reeside  
1  36.609972 -121.897399  Foam /  Reeside  
2  36.610020 -121.897404  Foam /  Reeside  
3  36.609972 -121.897399  Foam /  Reeside  
4  36.609972 -121.897399  Foam /  Reeside

### Filter with Lists and Unpacking

Queries can use the `isin` and take lists, as well as unpack lists. This nifty trick can keep the code clean.

Tables used:
1. `view.transitstacks`: some NTD related data, with each row being an operator
1. `gtfs_schedule_fact_daily_feed_stops`: stops associated with daily feed, subset to a particular date
1. `views.gtfs_schedule_dim_stops`: lat/lon for all stops, need to subset to interested stops

Here, the `transitstacks` table is filtered down to interested counties and certain columns. Using `isin` and unpacking a list (asterisk) is a quick way to do this.

In [3]:
# Subset to counties of interest
lossan_counties = ['San Luis Obispo', 'Santa Barbara', 'Ventura',
                  'Los Angeles', 'San Diego', 'Orange']

# List the columns to keep
info_cols = ['itp_id', 'transit_provider', 'ntd_id',
             'modes', 'county', 'legacy_ntd_id']

vehicle_cols = ['bus', 'articulated_bus', 'over_the_road_bus',
                'school_bus', 'trolleybus', 'vintage_historic_trolley',
                'streetcar']

paratransit_cols = ['van', 'cutaway', 'automobile',
                     'minivan', 'sport_utility_vehicle']

# transitstacks has a lot of columns, and we want to keep a fairly large subset of them
lossan_df = (tbls.views.transitstacks()
             # Collect to turn into pandas.DataFrame earlier for isin to work
             >> collect()
             >> filter(_.county.isin(lossan_counties))
             # Nifty way to unpack large list of columns
             >> select(*(info_cols + vehicle_cols + paratransit_cols))
            )

lossan_df.head()

itp_id              transit_provider ntd_id modes       county  \
0      352  UC San Diego Campus Shuttles   None    MB    San Diego   
2      353            USC Transportation   None    MB  Los Angeles   
6      202          Mission City Transit   None    DR  Los Angeles   
9      103  El Segundo Lunchtime Shuttle   None    MB  Los Angeles   
12      28           Balboa Island Ferry   None    FB       Orange   

   legacy_ntd_id bus articulated_bus over_the_road_bus school_bus trolleybus  \
0           None                                                               
2           None                                                               
6           None                                                               
9           None                                                               
12          None                                                               

   vintage_historic_trolley streetcar van cutaway automobile minivan  \
0                                                                      
2                                                                      
6                                                                      
9                                                                      
12                                                                     

   sport_utility_vehicle  
0                         
2                         
6                         
9                         
12

Next, we can query the `gtfs_schedule_fat_daily_feed_stops` to grab the stops for a particular day's feed. Those stops are then joined to `gtfs_schedule_dim_stops` to get the lat/lon attached. Use `isin` to further filter the dataframe and only keep operators in the counties of interest.

In [4]:
# Grab stops for that day
lossan_stops = (tbls.views.gtfs_schedule_fact_daily_feed_stops()
                >> filter(_.date == SELECTED_DATE)
                >> select(_.stop_key, _.date)
                # Merge with stop geom using stop_key
                >> inner_join(_,
                              (tbls.views.gtfs_schedule_dim_stops()
                               >> select(_.calitp_itp_id, _.stop_key, _.stop_id,
                                         _.stop_lat, _.stop_lon)
                              ), on='stop_key')
                >> collect()
                # Only grab the ITP IDs we're interested in
                >> filter(_.calitp_itp_id.isin(lossan_df.itp_id))
                # This is to sort in order for ITP_ID and stop_id
                >> arrange(_.calitp_itp_id, _.stop_id)
               ).reset_index(drop=True)

lossan_stops.head()

stop_key        date   stop_lat  calitp_itp_id    stop_lon  \
0 -3465164382055991093  2021-09-01  34.079011              6 -118.117346   
1 -4388330883484799862  2021-09-01  34.079282              6 -118.111602   
2 -3890148245714116244  2021-09-01  34.080835              6 -118.114294   
3  3725728967761887153  2021-09-01  34.085540              6 -118.116863   
4 -6069710332211186970  2021-09-01  34.085874              6 -118.117052   

   stop_id  
0  2619783  
1  2619784  
2  2619785  
3  2619787  
4  2619788